# evaluate performance along various axes of sentence complexity

## data

In [1]:
print("start")
import sys
import os

sys.path.append("./enigma-transformed/src")
sys.path.append("./src")
sys.path.append("../src")
sys.path.append("../../src")

start


In [3]:
available_eval_columns = [ # actually AAAAA these are all with the english model... after | there will be with the actual model 
    "unigram_js_divergence",  # 17602 old, 18204; german: 18248 ; czech: 18251 | de: 18303 ; cs: 18351 (almost); 18533
    "gpt2_tokens_per_char",  #  17659 old, 18206; german: 18249; czech: 18252 | de: 18304 ; cs: 18354 almost; 18532
    "gpt2_perplexity",  # 17604 old, 18207; german: ...; czech: | de: 19096 ; cs: 18361 (almost... but not quite); 18531
    "bigram_js_divergence",  # 17657, 18209; german: 18250 ; czech: 18258 | de: 18305 ; cs: 18486
    "depth_of_parse_tree", #17687, 221; german: 18293 ; czech:| de: 18306 ; cs: 18485
    "named_entities", #17688, 222; german: 18290 ; czech: | de: 18307 ; cs: 18484
    "pos_js_divergence", #17689 23; german: 18291 ; czech: | de: 18322 ; cs: 18483
    "pos_bigram_js_divergence" #17690  24; german: 18292 ; czech: | de: 18325 ; cs: 18482
]
eval_column = available_eval_columns[2]
lang='de'
print("eval_column:", eval_column)
dataset_max_len = 200


eval_column: unigram_js_divergence


In [4]:
import pandas as pd

data_path = f"news.2013.{lang}.trainlen.{dataset_max_len}.merged.csv"
data = pd.read_csv(data_path)
# print len
print("len:", len(data))

# filter out rows which 'lang' != True
data = data[data['lang'] == True]
print("len:", len(data))
# weird is false
data = data[data['weird'] == False]

# print len
print("len:", len(data))
print("data loaded")
# sort by eval column
data.sort_values(eval_column, inplace=True)
print("data sorted")


len: 898502
len: 850701
len: 847588
data loaded
data sorted


In [ ]:
# print the first 10 rows
print(f"{data.head(10)=}")
# last 10 rows
print(f"{data.tail(10)=}")

rows = {
    0.1: data.iloc[len(data) // 1000 * 999 - 1000 : len(data) // 1000 * 999],
    1: data.iloc[len(data) // 100 * 99 - 1000 : len(data) // 100 * 99],
    5: data.iloc[len(data) // 100 * 95 - 1000 : len(data) // 100 * 95],
    10: data.iloc[len(data) // 100 * 90 - 1000 : len(data) // 100 * 90],
    15: data.iloc[len(data) // 100 * 85 - 1000 : len(data) // 100 * 85],
    20: data.iloc[len(data) // 100 * 80 - 1000 : len(data) // 100 * 80],
    25: data.iloc[len(data) // 100 * 75 - 1000 : len(data) // 100 * 75],
    30: data.iloc[len(data) // 100 * 70 - 1000 : len(data) // 100 * 70],
    35: data.iloc[len(data) // 100 * 65 - 1000 : len(data) // 100 * 65],
    40: data.iloc[len(data) // 100 * 60 - 1000 : len(data) // 100 * 60],
    45: data.iloc[len(data) // 100 * 55 - 1000 : len(data) // 100 * 55],
    50: data.iloc[len(data) // 100 * 50 - 1000 : len(data) // 100 * 50],
    55: data.iloc[len(data) // 100 * 45 - 1000 : len(data) // 100 * 45],
    60: data.iloc[len(data) // 100 * 40 - 1000 : len(data) // 100 * 40],
    65: data.iloc[len(data) // 100 * 35 - 1000 : len(data) // 100 * 35],
    70: data.iloc[len(data) // 100 * 30 - 1000 : len(data) // 100 * 30],
    75: data.iloc[len(data) // 100 * 25 - 1000 : len(data) // 100 * 25],
    80: data.iloc[len(data) // 100 * 20 - 1000 : len(data) // 100 * 20],
    85: data.iloc[len(data) // 100 * 15 - 1000 : len(data) // 100 * 15],
    90: data.iloc[len(data) // 100 * 10 - 1000 : len(data) // 100 * 10],
    95: data.iloc[len(data) // 100 * 5 - 1000 : len(data) // 100 * 5],
    99: data.iloc[len(data) // 100 * 1 - 1000 : len(data) // 100 * 1],
    99.9: data.iloc[len(data) // 1000 * 1 - 1000 : len(data) // 1000 * 1],
}
if len(rows[99.9]) == 0: # bug for datasets with < 1M rows
    rows[99.9] = data.iloc[:1000]
else:
    print('if czech something weird happened, otherwise ok')
print("percentile rows split")

## load model

In [ ]:
from transformers import ByT5Tokenizer, T5ForConditionalGeneration
from src.utils import levensthein_distance, print_avg_median_mode_error
from transformers import pipeline, logging
import torch

logging.set_verbosity(logging.ERROR)


tokenizer = ByT5Tokenizer()

from src.ByT5Dataset import ByT5CaesarRandomDataset, ByT5ConstEnigmaDataset
# aaaaaa
model_ids = {
    'caesar': 'slurm_16677',
    'en': 'slurm_17510',
    'de': 'slurm_18065',
    'cs': 'slurm_18066'
}
model_id = model_ids[lang]
model = T5ForConditionalGeneration.from_pretrained(f"./logs/{model_id}/model")
dataset_class = ByT5ConstEnigmaDataset  # for 17510 model
# dataset_class = ByT5CaesarRandomDataset # for 16677 model
rows_datasets = {i: dataset_class(list(rows[i].text), dataset_max_len) for i in rows}

print("rows_datasets created")

averages, medians, modes = {}, {}, {}
raw_data = {}
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
for i, test in rows_datasets.items():
    print(f"#############################################")
    print(f"Testing {i}th percentile")
    error_counts = []
    translate = pipeline("translation", model=model, tokenizer=tokenizer, device=device)
    for index in range(len(test)):
        generated = translate(
            test[index]["input_text"], max_length=(dataset_max_len + 1) * 2
        )[0]["translation_text"]
        error_counts.append(levensthein_distance(generated, test[index]["output_text"]))
        if error_counts[-1] > 0:
            print(f"Example {index}, error count {error_counts[-1]}")
            print("In :", test[index]["input_text"])
            print("Gen:", generated)
            expected = test[index]["output_text"]
            print("Exp:", expected)
        else:
            print(f"Example {index} OK")
        print("-----------------------")

    avg, med, mode = print_avg_median_mode_error(error_counts)
    averages[i] = avg
    medians[i] = med
    modes[i] = mode
    raw_data[i] = error_counts

print("Averages:", averages)
print("Medians:", medians)
print("Modes:", modes)
print("Raw data:", raw_data)